# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,10.41,58,90,9.93,SH,1696201036
1,1,klyuchi,52.2667,79.1667,5.45,90,0,2.57,RU,1696201037
2,2,enterprise,36.0252,-115.2419,23.16,31,75,1.54,US,1696201037
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,3.33,100,100,4.00,RU,1696201037
4,4,adamstown,-25.0660,-130.1015,20.40,82,11,11.34,PN,1696201037


---

### Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    geo = True,
    tiles = "OSM",
    size = "Humidity"
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df['Max Temp']<25) & (city_data_df['Cloudiness']<25), :]


# Drop any rows with null values
narrow_city_df.dropna()

# Display sample data
narrow_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,klyuchi,52.2667,79.1667,5.45,90,0,2.57,RU,1696201037
4,4,adamstown,-25.0660,-130.1015,20.40,82,11,11.34,PN,1696201037
8,8,mantua,45.1667,10.7833,18.58,85,1,0.45,IT,1696201037
12,12,grytviken,-54.2811,-36.5092,3.46,79,1,5.01,GS,1696200837
17,17,hermanus,-34.4187,19.2345,12.96,73,3,3.50,ZA,1696201038


### Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = narrow_city_df[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,klyuchi,RU,52.2667,79.1667,90,
4,adamstown,PN,-25.0660,-130.1015,82,
8,mantua,IT,45.1667,10.7833,85,
12,grytviken,GS,-54.2811,-36.5092,79,
17,hermanus,ZA,-34.4187,19.2345,73,


### For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
klyuchi - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
mantua - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
tieling - nearest hotel: 金城粤海国际酒店
port lincoln - nearest hotel: Boston Hotel
yuzhno-kurilsk - nearest hotel: Dom Druzhby
grand falls-windsor - nearest hotel: Hotel Robin Hood
bredasdorp - nearest hotel: Victoria Hotel
znamenskoye - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
al bardiyah - nearest hotel: فندق البردي
ilha de mocambique - nearest hotel: Ruby Backpackers
marsabit - nearest hotel: Chicho Hotel
zuwarah - nearest hotel: فندق الدلفين
albany - nearest hotel: No hotel found
santa terezinha - nearest hotel: Pousada Santa Terezinha
morinville - nearest hotel: No hotel found
l'alcora - nearest hotel: No hotel found
lindi - nearest hotel: Baraka inn
honggang - nearest hotel: No hotel found
al jawf - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
1,klyuchi,RU,52.2667,79.1667,90,No hotel found
4,adamstown,PN,-25.0660,-130.1015,82,No hotel found
8,mantua,IT,45.1667,10.7833,85,No hotel found
12,grytviken,GS,-54.2811,-36.5092,79,No hotel found
17,hermanus,ZA,-34.4187,19.2345,73,Aloe guest house
...,...,...,...,...,...,...
550,saguenay,CA,48.4168,-71.0657,55,Le Montagnais
557,coos bay,US,43.3665,-124.2179,82,No hotel found
558,gobernador roca,AR,-27.1864,-55.4643,51,Residencial Doka
562,buy,RU,58.4786,41.5300,94,No hotel found


### Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot_2
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    geo = True,
    tiles = "OSM",
    size = "Humidity", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot_2
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)